# railファイル確認用

In [1]:
import boto3
import os
import shelve
import copy
import streamlit as st
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO
import pandas as pd
import numpy as np
import datetime
# import src.utilsST_01 as utlst
import src.helpers as helpers
import src.visualize as vis
from src.config import appProperties
from src.trolley import trolley
from src.similar_pixel import pixel
from pprint import pprint

config = appProperties('config.yml')

2023-06-20 07:45:35.077 INFO    numexpr.utils: NumExpr defaulting to 2 threads.


In [2]:
images_path = helpers.list_imagespath_nonCache(config.image_dir)
# dir_area = "Chuo_01_Tokyo-St_up_20230201_knight"
# dir_area = "Chuo_02_Tokyo-Kanda_up_20230201_knight"
# dir_area = "Kawagoe_02_Nishioomiya-Sashiougi_down_20220626_knight"
dir_area = "Chuo_24_Higashinakano-Nakano_down_20230201_day"
camera_num = "HD21"
target_dir = config.image_dir + "/" + dir_area + "/" + camera_num
outpath = config.output_dir + "/" + dir_area + "/" + camera_num

base_images = helpers.list_images(target_dir)

rail_fpath = outpath + "/rail.shelve"

# with shelve.open(rail_fpath) as rail:
#     rail_dict = copy.deepcopy(rail)

# rail = shelve.open(rail_fpath)

# shelveファイルの中身を1階層ずつチェックする

In [ ]:
keys = rail_dict.keys()
for i, key in enumerate(keys):
    print(f'{i}: {key}')

In [ ]:
keys = rail_dict[camera_num].keys()
for i, key in enumerate(keys):
    print(f'{i}: {key}')

In [ ]:
image_path = base_images[0]
image_path

In [ ]:
keys = rail_dict[camera_num][image_path].keys()
for i, key in enumerate(keys):
    print(f'{i}: {key}')

In [ ]:
keys = rail_dict[camera_num][image_path]["trolley1"].keys()
for i, key in enumerate(keys):
    # print(f'{i}: {key}')
    try:
        key_len = len(rail_dict[camera_num][image_path]["trolley1"][key])
    except:
        key_len = 0
    print(f'{key} > length:{key_len}')

In [ ]:
image_path = base_images[0]
rail["HD11"][image_path]["trolley1"]["estimated_upper_edge"]

In [ ]:
'''
case1: xのみnp.array   -> 2023-05-24 00:44:31.574638 Process end :0:00:45.240699
case2: x,yをnp.array   -> 2023-05-24 02:24:05.555617 Process end :0:00:57.180313
case3: rail_dict化     -> 2023-05-24 04:58:30.819891 Process end :0:00:07.624567
case4: shelve.open追加 -> 2023-05-24 05:17:12.809013 Process end :0:00:03.504746
'''

dt01 = datetime.datetime.now()

img_num = 50

with shelve.open(rail_fpath) as rail:
    trolley_dict = copy.deepcopy(rail[camera_num])

for idx, image_path in enumerate(base_images):
    print(f"{idx}> {image_path}")
    x_values = np.array([n + 1000 * idx for n in trolley_dict[image_path][config.trolley_ids[0]]["ix"]])
    for trolley_id in config.trolley_ids:
        plt.plot(x_values, trolley_dict[image_path][trolley_id]["estimated_upper_edge"])
        plt.plot(x_values, trolley_dict[image_path][trolley_id]["estimated_lower_edge"])
    if idx >= img_num-1:
        print("break")
        break

plt.show()

dt02 = datetime.datetime.now()
prc_time = dt02 - dt01
print(str(datetime.datetime.now()) + f' Process end :{prc_time}')

In [ ]:
for key in trolley_dict[base_images[0]]['trolley1'].keys():
    print(key)

In [ ]:
print("最後の検出結果")
for trolley_id in config.trolley_ids:
    print(f"trolley_id:{trolley_id}")
    print(f'estimated_upper_edge:{trolley_dict[image_path][trolley_id]["estimated_upper_edge"][999]}')
    print(f'estimated_lower_edge:{trolley_dict[image_path][trolley_id]["estimated_lower_edge"][999]}')
    print(f'brightness_std:{trolley_dict[image_path][trolley_id]["brightness_std"][999]}')
    print(f'brightness_center:{trolley_dict[image_path][trolley_id]["brightness_center"][999]}')

# 結果画像を表示する

In [ ]:
with shelve.open(rail_fpath) as rail:
    trolley_dict = copy.deepcopy(rail[camera_num])

idx = 0
image_path = base_images[idx]
img = Image.open(image_path)
print(f"{idx}> {image_path}")

x_values = np.array([n + 1000 * idx for n in trolley_dict[image_path][config.trolley_ids[0]]["ix"]])
for trolley_id in config.trolley_ids:
    plt.plot(x_values, trolley_dict[image_path][trolley_id]["estimated_upper_edge"])
    plt.plot(x_values, trolley_dict[image_path][trolley_id]["estimated_lower_edge"])


In [ ]:
# 画像とデータの読み込み
with shelve.open(rail_fpath) as rail:
    trolley_dict = copy.deepcopy(rail[camera_num])

idx = 0
image_path = base_images[idx]
img = Image.open(image_path)
print(f"{idx}> {image_path}")

# 画像をnumpy配列に変換
img_array = np.array(img)

# ランダムに1000画素を選択し、その平均輝度を背景の輝度とする
random_pixels = img_array[np.random.randint(0, img_array.shape[0], 1000), np.random.randint(0, img_array.shape[1], 1000)]
background_brightness = random_pixels.mean()

# データを描画
x_values = trolley_dict[image_path][config.trolley_ids[0]]["ix"]
for trolley_id in config.trolley_ids:
    upper_edge = trolley_dict[image_path][trolley_id]["estimated_upper_edge"]
    lower_edge = trolley_dict[image_path][trolley_id]["estimated_lower_edge"]
    for x, y1, y2 in zip(x_values, upper_edge, lower_edge):
        # estimated_upper_edgeとestimated_lower_edgeが0でない場合のみ色を変更
        if y1 != 0:
            color_upper = [0, 255, 0] if background_brightness < 128 else [255, 0, 0]  # 緑または赤
            img_array[y1, x] = color_upper
        if y2 != 0:
            color_lower = [0, 255, 0] if background_brightness < 128 else [255, 0, 0]  # 緑または赤
            img_array[y2, x] = color_lower

# 変更後の画像を表示
display_img = Image.fromarray(img_array)
display_img

# 結果をCSV出力する

In [ ]:
# CSVファイルの保存パスを指定
csv_fpath = rail_fpath.replace(".shelve", ".csv")

# shelveファイルを読み込む
with shelve.open(rail_fpath) as rail:
    trolley_dict = copy.deepcopy(rail[camera_num])

In [ ]:
base_images = list(trolley_dict.keys())
print(f"Images count: {len(base_images)}")
img_path = base_images[int(input("set index number: "))]
print(f"img_path: {img_path}")

In [ ]:
config.trolley_ids

In [ ]:
for key in trolley_dict[img_path][config.trolley_ids[0]].keys():
    print(key)

In [ ]:
trolley_id = config.trolley_ids[0]
key = "estimated_upper_edge"
print(trolley_id)
print(type(trolley_dict[img_path][trolley_id][key]))
print(len(trolley_dict[img_path][trolley_id][key]))
print(trolley_dict[img_path][trolley_id][key])

In [17]:
# 単体テスト用
# shelveをCSVファイルとして書き出す

# CSVファイルの保存パスを指定
csv_fpath = rail_fpath.replace(".shelve", ".csv")
print(f"csv_fpath:{csv_fpath}")

# 1画像あたりの間引く数を指定
thin_out = 100

# shelveファイルを読み込む
with shelve.open(rail_fpath) as rail:
    trolley_dict = copy.deepcopy(rail[camera_num])

dfs = pd.DataFrame()
for img_idx, img_path in enumerate(base_images):
    # print(f"{idx}> img_path: {img_path}")
    try:
        df_trolley = helpers.read_trolley_dict(config, trolley_dict, img_idx, img_path, thin_out).copy()
        # ixの値が連番になるように修正
        df_trolley['ix'] = df_trolley['ix'] + 1000 * img_idx
        dfs = pd.concat([dfs, df_trolley], ignore_index=True)
    except Exception as e:
        print("途中の画像まで変換できました。中断した画像👇")
        print(f"{img_idx}> img_path: {img_path}")
        print("エラーの内容👇")
        print(e)
        break
    # if idx > 10:
    #     break
# df_trolley
print("convert shelve to dataframe")
dfs

csv_fpath:output/Chuo_24_Higashinakano-Nakano_down_20230201_day/HD21/rail.csv
途中の画像まで変換できました。中断した画像👇
62> img_path: imgs/Chuo_24_Higashinakano-Nakano_down_20230201_day/HD21/2022_0613_HD21_01_00006886.jpg
エラーの内容👇
'trolley1'
convert shelve to dataframe


,img_idx,dir_area,camera_num,image_name,ix,trolley1_estimated_upper_edge,trolley1_estimated_lower_edge,trolley1_estimated_width,trolley1_estimated_slope,trolley1_brightness_center,...,trolley3_estimated_slope,trolley3_brightness_center,trolley3_brightness_mean,trolley3_brightness_std,trolley3_current_state,trolley3_estimated_upper_edge_variance,trolley3_estimated_lower_edge_variance,trolley3_estimated_slope_variance,trolley3_upper_boundary,trolley3_lower_boundary
0,0,Chuo_24_Higashinakano-Nakano_down_20230201_day,HD21,2022_0613_HD21_01_00006824.jpg,0,905,911,6,NaN,-1,...,NaN,11,11.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,0,Chuo_24_Higashinakano-Nakano_down_20230201_day,HD21,2022_0613_HD21_01_00006824.jpg,1,905,911,6,NaN,-4,...,NaN,11,11.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,0,Chuo_24_Higashinakano-Nakano_down_20230201_day,HD21,2022_0613_HD21_01_00006824.jpg,2,905,911,6,NaN,-1,...,NaN,12,12.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,0,Chuo_24_Higashinakano-Nakano_down_20230201_day,HD21,2022_0613_HD21_01_00006824.jpg,3,905,911,6,NaN,-1,...,NaN,12,12.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,0,Chuo_24_Higashinakano-Nakano_down_20230201_day,HD21,2022_0613_HD21_01_00006824.jpg,4,905,911,6,NaN,-1,...,NaN,13,13.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61995,61,Chuo_24_Higashinakano-Nakano_down_20230201_day,HD21,2022_0613_HD21_01_00006885.jpg,61995,748,753,5,NaN,-1,...,NaN,14,14.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
61996,61,Chuo_24_Higashinakano-Nakano_down_20230201_day,HD21,2022_0613_HD21_01_00006885.jpg,61996,748,753,5,NaN,-4,...,NaN,14,14.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
61997,61,Chuo_24_Higashinakano-Nakano_down_20230201_day,HD21,2022_0613_HD21_01_00006885.jpg,61997,748,753,5,NaN,-7,...,NaN,14,14.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
61998,61,Chuo_24_Higashinakano-Nakano_down_20230201_day,HD21,2022_0613_HD21_01_00006885.jpg,61998,748,754,6,NaN,-1,...,NaN,14,14.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
dfs.columns

Index(['img_idx', 'dir_area', 'camera_num', 'image_name', 'ix',
       'trolley1_estimated_upper_edge', 'trolley1_estimated_lower_edge',
       'trolley1_estimated_width', 'trolley1_estimated_slope',
       'trolley1_brightness_center', 'trolley1_brightness_mean',
       'trolley1_brightness_std', 'trolley1_current_state',
       'trolley1_estimated_upper_edge_variance',
       'trolley1_estimated_lower_edge_variance',
       'trolley1_estimated_slope_variance', 'trolley1_upper_boundary',
       'trolley1_lower_boundary', 'trolley2_estimated_upper_edge',
       'trolley2_estimated_lower_edge', 'trolley2_estimated_width',
       'trolley2_estimated_slope', 'trolley2_brightness_center',
       'trolley2_brightness_mean', 'trolley2_brightness_std',
       'trolley2_current_state', 'trolley2_estimated_upper_edge_variance',
       'trolley2_estimated_lower_edge_variance',
       'trolley2_estimated_slope_variance', 'trolley2_upper_boundary',
       'trolley2_lower_boundary', 'trolley3_estim

In [ ]:
# CSVファイルに変換する
dfs.to_csv(csv_fpath, encoding='cp932')

# CSVファイルからグラフを表示する

In [ ]:
from bokeh.plotting import figure, gridplot, output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.palettes import d3
output_notebook()


def plot_fig_bokeh(csv_fpath, graph_height, graph_width):

    # CSVファイルをデータフレームとして読み込む
    df_csv = pd.read_csv(csv_fpath, encoding='cp932')
    
    # CSVから作成したデータフレームをbokeh形式で読み込む
    source = ColumnDataSource(data=df_csv)
    
    # グラフの色情報を設定する
    # グラフの数に合わせて12色だけ取得する
    colors = d3["Category20"][12]

    # ツールチップを設定
    TOOLTIPS_EDGE=[
        ('index', '@img_idx'),
        ('image_name', '@image_name'),
        ('upper_edge1', '@trolley1_estimated_upper_edge'),
        ('lower_edge1', '@trolley1_estimated_lower_edge'),
        ('upper_edge2', '@trolley2_estimated_upper_edge'),
        ('lower_edge2', '@trolley2_estimated_lower_edge'),
        ('upper_edge3', '@trolley3_estimated_upper_edge'),
        ('lower_edge3', '@trolley3_estimated_lower_edge'),
    ]
    TOOLTIPS_WIDTH=[
        ('index', '@img_idx'),
        ('image_name', '@image_name'),
        ('estimated_width1', '@trolley1_estimated_width'),
        ('estimated_width2', '@trolley2_estimated_width'),
        ('estimated_width3', '@trolley3_estimated_width'),
    ]
    TOOLTIPS_BRIGHTNESS=[
        ('index', '@img_idx'),
        ('image_name', '@image_name'),
        ('brightness_center1', '@trolley1_brightness_center'),
        ('brightness_center2', '@trolley2_brightness_center'),
        ('brightness_center3', '@trolley3_brightness_center'),
    ]

    # グラフを作成
    p_edge = figure(
        title="Upper and Lower Edge",
        sizing_mode="stretch_width",
        tooltips=TOOLTIPS_EDGE,
        height=int(graph_height),
        width=int(graph_width)
    )
    p_width = figure(
        title="Width",
        sizing_mode="stretch_width",
        tooltips=TOOLTIPS_WIDTH,
        x_range=p_edge.x_range,
        height=int(graph_height),
        width=int(graph_width)
    )
    p_center = figure(
        title="Brightness Center",
        sizing_mode="stretch_width",
        tooltips=TOOLTIPS_BRIGHTNESS,
        x_range=p_edge.x_range,
        height=int(graph_height),
        width=int(graph_width)
    )

    # グラフを表示する領域を作成
    grid = gridplot(
        [[p_edge], [p_width], [p_center]],
        toolbar_location="above"
    )

    # データを追加
    x_values = "ix"
    upper_edge1 = "trolley1_estimated_upper_edge"
    lower_edge1 = "trolley1_estimated_lower_edge"
    upper_edge2 = "trolley2_estimated_upper_edge"
    lower_edge2 = "trolley2_estimated_lower_edge"
    upper_edge3 = "trolley3_estimated_upper_edge"
    lower_edge3 = "trolley3_estimated_lower_edge"
    estimated_width1 = "trolley1_estimated_width"
    estimated_width2 = "trolley2_estimated_width"
    estimated_width3 = "trolley3_estimated_width"
    brightness_center1 = "trolley1_brightness_center"
    brightness_center2 = "trolley2_brightness_center"
    brightness_center3 = "trolley3_brightness_center"

    # グラフにデータを追加
    # 1つ目のグラフ（Upper and Lower Edge）
    p_edge.line(x_values, upper_edge1, line_color=colors[0], source=source)
    p_edge.line(x_values, upper_edge2, line_color=colors[1], source=source)
    p_edge.line(x_values, upper_edge3, line_color=colors[2], source=source)
    p_edge.line(x_values, lower_edge1, line_color=colors[3], source=source)
    p_edge.line(x_values, lower_edge2, line_color=colors[4], source=source)
    p_edge.line(x_values, lower_edge3, line_color=colors[5], source=source)

    # 2つ目のグラフ（Brightness Std）
    p_width.line(x_values, estimated_width1, line_color=colors[6], source=source)
    p_width.line(x_values, estimated_width2, line_color=colors[7], source=source)
    p_width.line(x_values, estimated_width3, line_color=colors[8], source=source)

    # 3つ目のグラフ（Brightness Center）
    p_center.line(x_values, brightness_center1, line_color=colors[9], source=source)
    p_center.line(x_values, brightness_center2, line_color=colors[10], source=source)
    p_center.line(x_values, brightness_center3, line_color=colors[11], source=source)

    return grid

# メインループ
# CSVファイルの保存パスを指定
csv_fpath = rail_fpath.replace(".shelve", ".csv")
print(f"csv path:{csv_fpath}")

graph_height = 200
graph_width = 1000

grid = plot_fig_bokeh(csv_fpath, graph_height, graph_width)

show(grid)


In [ ]:
# グラフを作成
p_edge = figure(
    title="Upper and Lower Edge",
    sizing_mode="stretch_width",
    height=int(graph_height)
)
p_width = figure(
    title="Width",
    sizing_mode="stretch_width",
    x_range=p_edge.x_range,
    height=int(graph_height)
)
p_center = figure(
    title="Brightness Center",
    sizing_mode="stretch_width",
    x_range=p_edge.x_range,
    height=int(graph_height)
)

# グラフを表示する領域を作成
grid = gridplot(
    [[p_edge], [p_width], [p_center]],
    toolbar_location="above"
)

# データを追加
x_values = df_csv["ix"]
upper_edge = df_csv["trolley1_estimated_upper_edge"]
lower_edge = df_csv["trolley1_estimated_lower_edge"]
estimated_width = df_csv["trolley1_estimated_width"]
brightness_center = df_csv["trolley1_brightness_center"]

# グラフにデータを追加
# 1つ目のグラフ（Upper and Lower Edge）
p_edge.line(x_values, upper_edge, line_color="blue")
p_edge.line(x_values, lower_edge, line_color="red")

# 2つ目のグラフ（Brightness Std）
p_width.line(x_values, estimated_width, line_color="green")

# 3つ目のグラフ（Brightness Center）
p_center.line(x_values, brightness_center, line_color="orange")

show(grid)

In [ ]:
def set_column_names(config, trolley_dict):
    column_names = []
    for idx, trolley_id in enumerate(config.trolley_ids):
        for key in trolley_dict[img_path][trolley_id].keys():
            column_names.append(trolley_id + "_" + key)
    return column_names

column_names = set_column_names(config, trolley_dict)
pprint(f"column length: {len(column_names)}")
pprint(column_names)

In [ ]:
print(f"img_path: {img_path}")
for key in trolley_dict[img_path][config.trolley_ids[0]].keys():
    pprint(f"{key} > {len(trolley_dict[img_path][config.trolley_ids[0]][key])}")